In [1]:
import pandas as pd
import featuretools as ft

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
print(ft.list_primitives())


                                 name         type  dask_compatible  \
0                count_inside_nth_std  aggregation            False   
1                       n_unique_days  aggregation            False   
2                    time_since_first  aggregation            False   
3                            variance  aggregation            False   
4                    count_below_mean  aggregation            False   
5                               count  aggregation             True   
6           max_consecutive_positives  aggregation            False   
7                       n_most_common  aggregation            False   
8                                 any  aggregation             True   
9                                 max  aggregation             True   
10        is_monotonically_decreasing  aggregation            False   
11                                min  aggregation             True   
12       num_consecutive_greater_mean  aggregation            False   
13    

In [4]:
Customers = pd.DataFrame({
    'C_ID': ['C1', 'C2'],
    'Name': ['Martin', 'Julia'],
    'Creation_date': pd.to_datetime(['2018-08-15', '2020-05-05'])
})

Orders = pd.DataFrame({
    'Ord_ID': ['1', '2', '3', '4', '5'],
    'C_ID': ['C1', 'C2', 'C1', 'C1', 'C2']
})

Payments = pd.DataFrame({
    'Ord_ID': ['1', '5', '3', '4', '2'],
    'Price': [500, 200, 300, 100, 900]
})

Customers['Creation_date'] = pd.to_datetime(Customers['Creation_date'], format='%Y-%m-%d')

In [5]:
es = ft.EntitySet(id='Customer_Orders')


In [6]:
es = es.add_dataframe(
    dataframe_name='Customers',
    dataframe=Customers,
    index='C_ID',
    time_index='Creation_date')

es = es.add_dataframe(
    dataframe_name='Orders',
    dataframe=Orders,
    index='Ord_ID'
)


es = es.add_dataframe(
    dataframe_name='Payments',
    dataframe=Payments,
    make_index=True,
    index='payment_index')

c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parse

In [7]:
r_customer_orders = ft.Relationship(es, \
    parent_dataframe_name='Customers',
    parent_column_name='C_ID',
    child_dataframe_name='Orders',
    child_column_name='C_ID')

r_orders_payments = ft.Relationship(es, \
    parent_dataframe_name='Orders',
    parent_column_name='Ord_ID',
    child_dataframe_name='Payments',
    child_column_name='Ord_ID'
)

In [8]:
es.add_relationships([r_customer_orders, r_orders_payments])

Entityset: Customer_Orders
  DataFrames:
    Customers [Rows: 2, Columns: 3]
    Orders [Rows: 5, Columns: 2]
    Payments [Rows: 5, Columns: 3]
  Relationships:
    Orders.C_ID -> Customers.C_ID
    Payments.Ord_ID -> Orders.Ord_ID

In [9]:
print(es)

Entityset: Customer_Orders
  DataFrames:
    Customers [Rows: 2, Columns: 3]
    Orders [Rows: 5, Columns: 2]
    Payments [Rows: 5, Columns: 3]
  Relationships:
    Orders.C_ID -> Customers.C_ID
    Payments.Ord_ID -> Orders.Ord_ID


In [10]:
features, feature_names = ft.dfs(
    entityset=es,
    target_dataframe_name='Customers',
    agg_primitives=['sum'],
    trans_primitives=['year'],
    verbose=True  # If you want to see the progress output
)

Built 2 features
Elapsed: 00:00 | Progress: 100%|██████████


c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:828: FutureWarning: The provided callable <function sum at 0x00000170D375D260> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)


In [11]:
print(features)


      SUM(Payments.Price) YEAR(Creation_date)
C_ID                                         
C1                  900.0                2018
C2                 1100.0                2020


In [12]:
feats, feat_names = ft.dfs(
    entityset=es,
    target_dataframe_name='Customers',
    max_depth=2,
    verbose=True
)

Built 48 features
Elapsed: 00:00 | Progress: 100%|██████████


c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:828: FutureWarning: The provided callable <function std at 0x00000170D375E480> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:828: FutureWarning: The provided callable <function max at 0x00000170D375D940> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ).agg(to_agg)
c:\Users\mrosk\.conda\envs\ml311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:828: FutureWarning: The provided callable <function sum at 0x00000170D375D260> is currently using SeriesGroupBy.sum. In a future ve

In [13]:
print(feats)

      COUNT(Orders)  COUNT(Payments)  MAX(Payments.Price)  \
C_ID                                                        
C1                3                3                500.0   
C2                2                2                900.0   

      MEAN(Payments.Price)  MIN(Payments.Price)  SKEW(Payments.Price)  \
C_ID                                                                    
C1                   300.0                100.0                   0.0   
C2                   550.0                200.0                   NaN   

      STD(Payments.Price)  SUM(Payments.Price) DAY(Creation_date)  \
C_ID                                                                
C1             200.000000                900.0                 15   
C2             494.974747               1100.0                  5   

     MONTH(Creation_date) WEEKDAY(Creation_date) YEAR(Creation_date)  \
C_ID                                                                   
C1                      8                